### BEFORE PART A

In [46]:
#installing necessary packages

!pip install simpletransformers
!pip install checklist
!jupyter nbextension install --py --sys-prefix checklist.viewer
!jupyter nbextension enable --py --sys-prefix checklist.viewer

  Using cached checklist-0.0.11.tar.gz (12.1 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached patternfork_nosql-3.6.tar.gz (22.3 MB)
  Using cached iso-639-0.4.5.tar.gz (167 kB)
     |████████████████████████████████| 81 kB 6.2 MB/s 
     |████████████████████████████████| 5.6 MB 20.7 MB/s 
     |████████████████████████████████| 5.6 MB 51.3 MB/s 
     |████████████████████████████████| 419 kB 45.3 MB/s 
     |████████████████████████████████| 104 kB 49.9 MB/s 
     |████████████████████████████████| 3.6 MB 42.8 MB/s 
  Created wheel for checklist: filename=checklist-0.0.11-py3-none-any.whl size=12165635 sha256=064c4a3b482f90f0590a87e220bd289e28105da4d0b4508b2a33efee98d76d2f
  Stored in directory: /root/.cache/pip/wheels/6a/8a/07/6446879be434879c27671c83443727d74cecf6b630c8a24d03
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169061 sha256=6a97764655928bcbfe37d35f5cffecadc3a3015c6943bcfe4dbca7573e79c15c
  Stored in directory: /root/.

In [66]:
#importing necessary packages 

import csv
from random import randint
import numpy as np
import simpletransformers
from simpletransformers.classification import ClassificationModel
import pandas as pd
import pickle
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
import spacy
nlp = spacy.load('en_core_web_sm')

### PART A

1.  Class distributions 

In [5]:


#Load the training set 

with open('/content/olid-train.csv') as train_file:
    reader = csv.reader(train_file)


#Number of offensive and not offensive instances in training set

    counter1 = 0 #offensive
    counter0 = 0 #not offensive

    for row in reader:
        if row[2] == "1":
            counter1 += 1 
        if row[2] == "0":
            counter0 += 1 

print(str(counter1) + " offensive")
print(str(counter0) + " not offensive")

4400 offensive
8840 not offensive


In [ ]:
#Relative label frequency (%)

print(str(round((counter1/(counter1+counter0))*100,2)) + " offensive")
print(str(round((counter0/(counter1+counter0))*100,2)) + " not offensive")

33.23 offensive
66.77 not offensive


2. Baselines

In [13]:
#Functions necessary to calculate precision, recall, F1 and weighted F1

def prec(TP, FP):
    try:
        precision = TP/(TP+FP)
    except ZeroDivisionError:
        precision = 0
    return precision

def recall(TP, FN):
    try:
        rec = TP/(TP+FN)
    except ZeroDivisionError:
        rec = 0
    return rec

def f1_mes(prec, recall):
    try:
        f1 = 2*prec*recall/(prec+recall)
    except ZeroDivisionError:
        f1 = 0
    return f1

def weighted_f1(n_c, f1_c, n_n, f1_n):
    try:
        w_f1 = (n_c*f1_c + n_n*f1_n)/(n_c+n_n)
    except ZeroDivisionError:
        w_f1 = 0
    return w_f1

In [11]:
#Function for majority baseline

maj_base = '0'

def majority_baseline(test_set, maj_baseline = '0'):
    decisions = []
    correct_1 = 0
    correct_0 = 0
    false_1 = 0
    false_0 = 0
    for row in test_set:
        decisions.append(maj_baseline)
        if maj_baseline == row[2] == '1':
            correct_1 += 1
        elif maj_baseline == row[2] == '0':
            correct_0 +=1
        elif maj_baseline == '1'  and row[2] == '0':
            false_1 += 1
        elif maj_baseline == '0' and row[2] == '1':
            false_0 +=1
    
    prec_1 = prec(correct_1, false_1)
    prec_0 = prec(correct_0,false_0)

    recall_1 = recall(correct_1, false_0)
    recall_0 = recall(correct_0,false_1)

    f1_1 = f1_mes(prec_1, recall_1)
    f1_0 = f1_mes(prec_0, recall_0)

    macro_prec = np.mean([prec_1, prec_0])
    macro_recall = np.mean([recall_1, recall_0])
    macro_f1 = np.mean([f1_1, f1_0])

    weigh_prec = (prec_1*counter1 + prec_0*counter0)/(counter1+counter0)
    weigh_rec = (recall_1*counter1 + recall_0*counter0)/(counter1+counter0)
    weigh_f1 = (f1_1*counter1 + f1_0*counter0)/(counter1+counter0)



    d = {"precision 1": prec_1, "precision 0": prec_0, 
         "recall 1": recall_1, "recall 0": recall_0, 
         "f1 1": f1_1, "f1 0": f1_0, 
         "macro prec": macro_prec, "macro recall": macro_recall, 
         "macro_f1": macro_f1, "weighted precision": weigh_prec, 
         "weighted recall": weigh_rec, "weighted f1": weigh_f1 }

    return d

In [10]:
#Function that generates random class '1' or '0'

def random_func():
    dec = randint(0,1)
    if dec == 1:
        random_class = "1"        
    else:
        random_class = "0"
    return random_class

In [8]:
#Function for random baseline

def random_baseline(test_set):
    decisions = []
    correct_1 = 0
    correct_0 = 0
    false_1 = 0
    false_0 = 0
    
    for row in test_set:
        rand_baseline = random_func()
        decisions.append(rand_baseline)
        if rand_baseline == row[2] == '1':
            correct_1 += 1
        elif rand_baseline == row[2] == '0':
            correct_0 +=1
        elif rand_baseline == '1'  and row[2] == '0':
            false_1 += 1
        elif rand_baseline == '0' and row[2] == '1':
            false_0 +=1
    
    prec_1 = prec(correct_1, false_1)
    prec_0 = prec(correct_0,false_0)

    recall_1 = recall(correct_1, false_0)
    recall_0 = recall(correct_0,false_1)

    f1_1 = f1_mes(prec_1, recall_1)
    f1_0 = f1_mes(prec_0, recall_0)
    
    macro_prec = np.mean([prec_1, prec_0])
    macro_recall = np.mean([recall_1, recall_0])
    macro_f1 = np.mean([f1_1, f1_0])

    weigh_prec = (prec_1*counter1 + prec_0*counter0)/(counter1+counter0)
    weigh_rec = (recall_1*counter1 + recall_0*counter0)/(counter1+counter0)
    weigh_f1 = (f1_1*counter1 + f1_0*counter0)/(counter1+counter0)



    d = {"precision 1": prec_1, "precision 0": prec_0, 
         "recall 1": recall_1, "recall 0": recall_0, 
         "f1 1": f1_1, "f1 0": f1_0, 
         "macro prec": macro_prec, "macro recall": macro_recall,
         "macro_f1": macro_f1, "weighted precision": weigh_prec, 
         "weighted recall": weigh_rec, "weighted f1": weigh_f1 }

    return d



In [14]:
#Load the test set

with open('/content/olid-test.csv') as test_file:
    reader = csv.reader(test_file)
    test_tweets = []
    for row in reader:
        test_tweets.append(row)
        
#Delete column names  

del test_tweets[0]


#majority and random baseline of the test tweets   

dict_maj = majority_baseline(test_tweets, '0')
dict_rand = random_baseline(test_tweets)

print(f'Majority = {dict_maj}')
print(f'Random = {dict_rand}')

   

Majority = {'precision 1': 0, 'precision 0': 0.7209302325581395, 'recall 1': 0.0, 'recall 0': 1.0, 'f1 1': 0, 'f1 0': 0.8378378378378378, 'macro prec': 0.36046511627906974, 'macro recall': 0.5, 'macro_f1': 0.4189189189189189, 'weighted precision': 0.4813461673575493, 'weighted recall': 0.6676737160120846, 'weighted f1': 0.5594023026047196}
Random = {'precision 1': 0.2831050228310502, 'precision 0': 0.7251184834123223, 'recall 1': 0.5166666666666667, 'recall 0': 0.4935483870967742, 'f1 1': 0.36578171091445427, 'f1 0': 0.5873320537428023, 'macro prec': 0.5041117531216863, 'macro recall': 0.5051075268817204, 'macro_f1': 0.47655688232862825, 'weighted precision': 0.5782257925847093, 'weighted recall': 0.5012311990384303, 'weighted f1': 0.5137050515944087}


3. Fine Tuning BERT

In [19]:
#Creating BERT model

#new_model =  ClassificationModel('bert', 'bert-base-cased') 

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [17]:
#Importing both dataframes ready for model evaluation

#with open('/content/olid-train.csv') as train_file:
#  train_df = pd.read_csv(train_file)
#with open('/content/olid-test.csv') as test_file:
#  test_df = pd.read_csv(test_file)

In [20]:
#Training model on the train dataframe

#new_model.train_model(train_df)

  0%|          | 0/13240 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1655 [00:00<?, ?it/s]

(1655, 0.5014528700952443)

In [23]:
#Saving trained model

#saved_model = 'saved_model.sav'
#pickle.dump(new_model, open(saved_model, 'wb'))

In [24]:
#Load trained model as 'model'

model = pickle.load(open(saved_model, 'rb'))

In [25]:
#Function that prints information about evaluation

def evaluation(result, model_output, wrong_prediction):
    print(f'Result:\n{result}\n')
    print(f'Model outputs\n{model_outputs}\n')
    print(f'Wrong predictions\n{wrong_predictions}')
  

In [63]:
#Function that provides all results of an evaluation

def results(tp, tn, fp, fn):
  
  print('\nMore Info:\n')
  #precisions of model on test set 
  prec1 = prec(tp, fp)
  prec0 = prec(tn, fn)
  print(f'precision 1 : {prec1}')
  print(f'precision 0 : {prec0}')

  #recalls of model on test set
  recall1 = recall(tp, fn)
  recall0  = recall(tn, fp)
  print(f'Recall 1 : {recall1}')
  print(f'Recall 0 : {recall0}')

  #F1s of model on test set
  f11 = f1_mes(prec1, recall1)
  f10 = f1_mes(prec0, recall0)
  print(f'F1 1 : {f11}')
  print(f'F1 0 : {f10}')  

  #Macros and weights of model on test set

  macro_prec = np.mean([prec1, prec0])
  macro_recall = np.mean([recall1, recall0])
  macro_f1 = np.mean([f11, f10])
  weigh_prec = (prec1*counter1 + prec0*counter0)/(counter1+counter0)
  weigh_rec = (recall1*counter1 + recall0*counter0)/(counter1+counter0)
  weigh_f1 = (f11*counter1 + f10*counter0)/(counter1+counter0)
  print(f'macro_prec : {macro_prec}')
  print(f'macro_recall : {macro_recall}')
  print(f'macro_f1 : {macro_f1}')
  print(f'weigh_prec : {weigh_prec}')
  print(f'weigh_rec : {weigh_rec}')
  print(f'weigh_f1 : {weigh_f1}')



In [32]:
#Number of offensive and not offensive instances in test set

with open('/content/olid-test.csv') as test_file:
    reader = csv.reader(test_file)

    counter1 = 0 #offensive
    counter0 = 0 #not offensive

    for row in reader:
      if row[2] == "1":
          counter1 += 1 
      if row[2] == "0":
          counter0 += 1 
print(str(counter1) + " offensive")
print(str(counter0) + " not offensive")

240 offensive
620 not offensive


In [62]:
#Evaluate test dataframe with trained model

result, model_outputs, wrong_predictions = model.eval_model(test_df)
evaluation(result, model_outputs, wrong_predictions)
results(result['tp'], result['fp'], result['fp'], result['fn'])

  0%|          | 0/860 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

Result:
{'mcc': 0.6037694378920293, 'tp': 156, 'tn': 572, 'fp': 48, 'fn': 84, 'auroc': 0.8733333333333333, 'auprc': 0.7487986357442238, 'eval_loss': 0.38618444513391564}

Model outputs
[[ 0.55957031  0.07336426]
 [ 0.49731445  0.31176758]
 [ 1.55761719 -0.50683594]
 ...
 [-1.30664062  0.46362305]
 [-0.70458984  0.21069336]
 [ 2.109375   -0.91845703]]

Wrong predictions
[{'guid': 0, 'text_a': '#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA Democrats Support Antifa, Muslim Brotherhood, MS13, ISIS, Pedophilia, Child Trafficking, Taxpayer Funded Abortion’s, Election Fraud, Sedition And Treason!!! #LockThemAllUp #WWG1WGA #QAnon \u2066@USER URL', 'text_b': None, 'label': 1}, {'guid': 4, 'text_a': '#NoPasaran: Unity demo to oppose the far-right in #London – #antifa #Oct13 — Enough is Enough! URL', 'text_b': None, 'label': 1}, {'guid': 6, 'text_a': '#RAP is a form of ART! Used to express yourself freely. It does not gv the green light or excuse the behavior of acting like an animal! She is not

# Inspect the tokenization of the OLIDv1 training set using the BERT’s tokenizer 

In [37]:
#Tokenizing text in train set
tokenized = []
for i in train_df['text']:
  tokenized.append(model.tokenizer.tokenize(i))


In [39]:
#Counting how many tokens, how many tokens are divided in subwords 

counter_tokens = 0
counter_split_tokens= 0
counter_subtokens = [] #subwords per token (with at least 2 subtokens)

for i in tokenized: 
  for j in range(len(i)):
    try:
      if '##' not in i[j] and '##' in i[j+1]:
        c = 1
        try: 
         while '##' in i[j+c]:
          c+=1
        except IndexError:
          counter_subtokens.append(c)
          continue
        counter_subtokens.append(c)
        counter_tokens+=1
        counter_split_tokens+=1
      elif '##' not in i[j] and '##' not in i[j+1]:
        counter_tokens+=1
      elif '##' in i[j]:
        continue
    except IndexError:
      continue
        
        
print(f'Tokens : {counter_tokens}')
print(f'Split tokens : {counter_split_tokens}')



Tokens : 374691
Split tokens : 63630


In [41]:
#Calculating mean of subwords per token

single_tokens = counter_tokens - counter_split_tokens
total_subtokens = []

for i in range(single_tokens):
  total_subtokens.append(1)

total_subtokens += counter_subtokens

print(f'Mean : {np.mean(total_subtokens)}')

Mean : 1.2407367246221959


In [42]:
#loading BERTs fixed vocabolary

vals = model.tokenizer.vocab.keys()

In [43]:
#Longest subword and its lenght

print(max(vals, key = len))
print(len(max(vals, key = len)))

telecommunications
18


In [ ]:
#!pip install regex

In [53]:
#Loading diagnostic test set

with open('/content/olid-subset-diagnostic-tests.csv') as train_file_100:
  original_df = pd.read_csv(train_file_100)
  typos_df = original_df
  

In [54]:
#Checking labels in diagnostic test set

original_df['labels'].value_counts()


1    50
0    50
Name: labels, dtype: int64

In [ ]:
counter1 = 50
counter0 = 50

In [71]:
#Evaluate diagnostic test dataframe with trained model

result, model_outputs, wrong_predictions = model.eval_model(original_df)
evaluation(result, model_outputs, wrong_predictions)
results(result['tp'], result['fp'], result['fp'], result['fn'])

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Result:
{'mcc': 0.6531972647421809, 'tp': 36, 'tn': 46, 'fp': 4, 'fn': 14, 'auroc': 0.8744, 'auprc': 0.8763334968585492, 'eval_loss': 0.45772713881272536}

Model outputs
[[-1.41601562  0.50244141]
 [ 1.57714844 -0.49389648]
 [-1.4375      0.52392578]
 [ 0.10668945  0.40551758]
 [-1.45507812  0.53710938]
 [-0.91357422  0.33178711]
 [ 2.14453125 -0.98632812]
 [-0.89746094  0.24072266]
 [ 1.54003906 -0.44702148]
 [-1.4765625   0.54931641]
 [-0.98730469  0.19702148]
 [-1.52441406  0.55371094]
 [-1.5625      0.55859375]
 [ 0.40454102  0.515625  ]
 [-1.32324219  0.46484375]
 [ 0.1628418   0.50683594]
 [-1.52929688  0.55664062]
 [-0.66015625  0.40307617]
 [-1.51074219  0.54248047]
 [-0.91162109  0.24780273]
 [-0.91992188  0.21325684]
 [-1.35449219  0.48217773]
 [-0.62744141  0.33447266]
 [ 0.37109375  0.38696289]
 [ 1.94726562 -0.65771484]
 [-0.87011719  0.3359375 ]
 [ 1.48242188 -0.42675781]
 [-1.41015625  0.51611328]
 [-1.53027344  0.56347656]
 [-0.3425293   0.21984863]
 [ 0.40795898  0.319

In [68]:
#Converting text in diagnostic test set in list of strings

pdata =list(nlp.pipe(original_df['text']))
for i in pdata:
  i = str(i)

In [72]:
np.random.seed(42) 

In [73]:
#Add 3 typos per tweet

pdata_typos = Perturb.perturb(typos_df['text'], Perturb.add_typos, keep_original=False, typos=3)
for i in range(len(pdata_typos['data'])):
  typos_df['text'][i] = pdata_typos['data'][i]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [74]:
#Evaluate typos dataframe with trained model

result, model_outputs, wrong_predictions = model.eval_model(typos_df)
evaluation(result, model_outputs, wrong_predictions)
results(result['tp'], result['fp'], result['fp'], result['fn'])

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Result:
{'mcc': 0.5768599504012304, 'tp': 33, 'tn': 45, 'fp': 5, 'fn': 17, 'auroc': 0.8320000000000001, 'auprc': 0.8491263358244565, 'eval_loss': 0.5241294273963342}

Model outputs
[[-0.79638672  0.21813965]
 [ 0.88085938 -0.10852051]
 [-1.40429688  0.50976562]
 [ 0.80810547 -0.07275391]
 [-1.34863281  0.49633789]
 [ 0.18481445  0.63916016]
 [ 2.09570312 -0.88671875]
 [-0.77539062  0.23486328]
 [ 1.68066406 -0.50244141]
 [-1.45898438  0.54345703]
 [-0.90136719  0.1574707 ]
 [-1.44042969  0.5234375 ]
 [ 1.47265625 -0.40673828]
 [ 0.4699707   0.30224609]
 [-1.26660156  0.4440918 ]
 [ 0.05279541  0.36621094]
 [-0.50537109  0.29370117]
 [ 0.11566162  0.62402344]
 [-1.46191406  0.52929688]
 [-0.8515625   0.28491211]
 [-0.88085938  0.21203613]
 [-1.26660156  0.44335938]
 [ 0.27148438  0.56884766]
 [ 0.28564453  0.53515625]
 [ 1.90429688 -0.62255859]
 [-0.90820312  0.29199219]
 [ 1.58691406 -0.48193359]
 [-1.35058594  0.4921875 ]
 [-1.43359375  0.52392578]
 [-0.99560547  0.25097656]
 [-1.2080

NEGATION


In [75]:
#Loading Diagnostic Test set for Negation
with open('/content/olid-subset-diagnostic-tests.csv') as train_file_100:
  negation_df = pd.read_csv(train_file_100)
 

In [78]:
#Add negation to Diagnostic Test set

pdata_neg = Perturb.perturb(pdata, Perturb.add_negation, keep_original=False)
for i in range(len(pdata_neg['data'])):
  negation_df['text'][i] = pdata_neg['data'][i]
  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [79]:
#Predicting label for first 10 negated tweets

for i in range(10):
  predict, raw = model.predict(pdata_neg['data'][i])
  print(predict)
  print(pdata_neg['data'][i])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]
["@USER @USER Who the hell doesn't he think he is?"]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

[0]
["#Democrats #Liberals you are being #threatened by #Armed #Nazis and you talk about #disarmament of yourselves and of #Disconnection from the #Military #Lifestyle ?  You shouldn't #broadcast your #Armory and #killSkills #RightFuckingNOW URL"]


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

[1]
["699. Just didn't want to tell you you should Hang Out With Me More. i think it’ll be fun! Also, lmao dick for a tongue? So people can easily suck me off. HAHA kidding aside, i aint want a tongue as my dick that looks hella weird.. but my choice was weird to but WHATEVER"]


In [80]:
#Evaluate negation dataframe with trained model

result, model_outputs, wrong_predictions = model.eval_model(negation_df)
evaluation(result, model_outputs, wrong_predictions)
results(result['tp'], result['fp'], result['fp'], result['fn'])

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

Result:
{'mcc': 0.625, 'tp': 33, 'tn': 47, 'fp': 3, 'fn': 17, 'auroc': 0.87, 'auprc': 0.8815130042574961, 'eval_loss': 0.47946959275465745}

Model outputs
[[-0.99560547  0.30737305]
 [ 1.43945312 -0.43334961]
 [-1.38867188  0.50732422]
 [ 0.19458008  0.57128906]
 [-1.34960938  0.49584961]
 [-0.76611328  0.23852539]
 [ 2.11132812 -0.90380859]
 [-0.68847656  0.24365234]
 [ 1.59472656 -0.46337891]
 [-1.44238281  0.53564453]
 [-0.9375      0.22229004]
 [-1.45703125  0.53369141]
 [-1.5078125   0.546875  ]
 [ 0.50927734  0.33569336]
 [-1.26757812  0.44165039]
 [ 0.18017578  0.4465332 ]
 [-1.41992188  0.52246094]
 [ 0.13793945  0.58740234]
 [-1.48632812  0.53857422]
 [-0.63476562  0.29150391]
 [-0.85498047  0.20239258]
 [-1.27246094  0.44750977]
 [-0.34667969  0.72558594]
 [ 0.41259766  0.34423828]
 [ 1.87988281 -0.60986328]
 [-0.30834961  0.17138672]
 [ 1.68847656 -0.52587891]
 [-1.34667969  0.49121094]
 [-1.47753906  0.54345703]
 [ 0.06646729  0.51660156]
 [ 0.44091797  0.32543945]
 [ 0.826

LAST PART


In [81]:
#Creating negated examples 

editor = Editor()
hate_mask = editor.template('I hate {mask}.', remove_duplicates=True)
nohate_mask= editor.template('I don’t hate {mask}', remove_duplicates=True)

hate_religion = editor.template('I hate {religion}', remove_duplicates = True)
nohate_religion = editor.template('I  don’t hate {religion}', remove_duplicates = True)

hate_nationality = editor.template('I hate {religion}', remove_duplicates = True)
nohate_nationality = editor.template('I  don’t hate {religion}', remove_duplicates = True)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/checklist/text_generation.py:171: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  to_pred = torch.tensor(to_pred, device=self.device).to(torch.int64)


In [86]:
#Adding all negated examples together

text = hate_mask + nohate_mask + hate_religion + nohate_religion + hate_nationality + nohate_nationality

#Converting examples to dataframe and save as file

dict = {'text': text}  
with open('hate_nohate.csv', 'w', newline='') as csvfile:
    fieldnames = ['sentence']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(dict['text']['data'])):
      writer.writerow({'sentence': dict['text']['data'][i] })
       




In [87]:
predictions, raw_outputs = model.predict(text['data'])

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/32 [00:00<?, ?it/s]

In [88]:
#predicting labels of negated examples with model

text = text['data']

for i in range(len(text)):
  print(f'{text[i]} is {predictions[i]}')

I hate it. is 1
I hate them. is 1
I hate capitalism. is 1
I hate cats. is 1
I hate politics. is 1
I hate you. is 1
I hate him. is 1
I hate people. is 1
I hate that. is 1
I hate Trump. is 1
I hate racism. is 1
I hate this. is 1
I hate robots. is 1
I hate spiders. is 1
I hate her. is 1
I hate death. is 1
I hate hate. is 1
I hate dogs. is 1
I hate war. is 1
I hate change. is 1
I hate men. is 1
I hate math. is 1
I hate women. is 1
I hate America. is 1
I hate feminism. is 1
I hate snakes. is 1
I hate animals. is 1
I hate technology. is 1
I hate voting. is 1
I hate pizza. is 1
I hate science. is 1
I hate bullies. is 1
I hate government. is 1
I hate Twitter. is 1
I hate myself. is 1
I hate religion. is 1
I hate zombies. is 1
I hate Obamacare. is 1
I hate divorce. is 1
I hate socialism. is 1
I hate discrimination. is 1
I hate violence. is 1
I hate liberals. is 1
I hate money. is 1
I hate guns. is 1
I hate me. is 1
I hate Nazis. is 1
I hate immigrants. is 1
I hate Facebook. is 1
I hate sex. is 